In [55]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re

import matplotlib.pyplot as plt

In [56]:
nltk.download("stopwords")
%matplotlib inline
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
pd.options.plotting.backend = "plotly"

Num GPUs Available:  1


[nltk_data] Downloading package stopwords to /home/szymon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
df0 = pd.read_csv("trumptweets.csv")
df = df0[["content", "retweets", "favorites"]].copy()
df

,content,retweets,favorites
0,Be sure to tune in and watch Donald Trump on L...,500,868
1,Donald Trump will be appearing on The View tom...,33,273
2,Donald Trump reads Top Ten Financial Tips on L...,12,18
3,New Blog Post: Celebrity Apprentice Finale and...,11,24
4,"""My persona will never be that of a wallflower...",1399,1965
...,...,...,...
41117,I have never seen the Republican Party as Stro...,32620,213817
41118,Now Mini Mike Bloomberg is critical of Jack Wi...,36239,149571
41119,I was thrilled to be back in the Great State o...,16588,66944
41120,"“In the House, the President got less due proc...",20599,81921


In [58]:
def tweet_to_words(tweet):
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [59]:
df["words"] = [tweet_to_words(item) for item in tqdm(df["content"])]
df

100%|██████████| 41122/41122 [00:49<00:00, 822.93it/s] 


,content,retweets,favorites,words
0,Be sure to tune in and watch Donald Trump on L...,500,868,"[sure, tune, watch, donald, trump, late, night..."
1,Donald Trump will be appearing on The View tom...,33,273,"[donald, trump, appear, view, tomorrow, morn, ..."
2,Donald Trump reads Top Ten Financial Tips on L...,12,18,"[donald, trump, read, top, ten, financi, tip, ..."
3,New Blog Post: Celebrity Apprentice Finale and...,11,24,"[new, blog, post, celebr, apprentic, final, le..."
4,"""My persona will never be that of a wallflower...",1399,1965,"[persona, never, wallflow, rather, build, wall..."
...,...,...,...,...
41117,I have never seen the Republican Party as Stro...,32620,213817,"[never, seen, republican, parti, strong, unifi..."
41118,Now Mini Mike Bloomberg is critical of Jack Wi...,36239,149571,"[mini, mike, bloomberg, critic, jack, wilson, ..."
41119,I was thrilled to be back in the Great State o...,16588,66944,"[thrill, back, great, state, texa, tonight, pe..."
41120,"“In the House, the President got less due proc...",20599,81921,"[hous, presid, got, less, due, process, 9, 11,..."


In [60]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = df[label].apply(lambda x: sid.polarity_scores(" ".join(x)))
    df["vader_neg"] = polarity_scores.apply(lambda x: x["neg"])
    df["vader_neu"] = polarity_scores.apply(lambda x: x["neu"])
    df["vader_pos"] = polarity_scores.apply(lambda x: x["pos"])
    df["vader_comp"] = polarity_scores.apply(lambda x: x["compound"])
    df["sentance"] = df[label].apply(lambda x: " ".join(x))
    return df

In [61]:
df2 = compute_vader_scores(df, "words")
df2

,content,retweets,favorites,words,vader_neg,vader_neu,vader_pos,vader_comp,sentance
0,Be sure to tune in and watch Donald Trump on L...,500,868,"[sure, tune, watch, donald, trump, late, night...",0.000,0.745,0.255,0.4767,sure tune watch donald trump late night david ...
1,Donald Trump will be appearing on The View tom...,33,273,"[donald, trump, appear, view, tomorrow, morn, ...",0.000,0.652,0.348,0.7506,donald trump appear view tomorrow morn discuss...
2,Donald Trump reads Top Ten Financial Tips on L...,12,18,"[donald, trump, read, top, ten, financi, tip, ...",0.000,0.893,0.107,0.2023,donald trump read top ten financi tip late sho...
3,New Blog Post: Celebrity Apprentice Finale and...,11,24,"[new, blog, post, celebr, apprentic, final, le...",0.000,1.000,0.000,0.0000,new blog post celebr apprentic final lesson le...
4,"""My persona will never be that of a wallflower...",1399,1965,"[persona, never, wallflow, rather, build, wall...",0.000,1.000,0.000,0.0000,persona never wallflow rather build wall cling...
...,...,...,...,...,...,...,...,...,...
41117,I have never seen the Republican Party as Stro...,32620,213817,"[never, seen, republican, parti, strong, unifi...",0.000,0.508,0.492,0.7003,never seen republican parti strong unifi right...
41118,Now Mini Mike Bloomberg is critical of Jack Wi...,36239,149571,"[mini, mike, bloomberg, critic, jack, wilson, ...",0.324,0.526,0.150,-0.7650,mini mike bloomberg critic jack wilson save pe...
41119,I was thrilled to be back in the Great State o...,16588,66944,"[thrill, back, great, state, texa, tonight, pe...",0.041,0.541,0.419,0.9325,thrill back great state texa tonight peopl kno...
41120,"“In the House, the President got less due proc...",20599,81921,"[hous, presid, got, less, due, process, 9, 11,...",0.227,0.773,0.000,-0.6908,hous presid got less due process 9 11 terroris...


In [62]:
def get_label_value(x: float):
    match x:
        case _ if x >= -0.2 and x <= 0.2:
            return 1
        case _ if x > 0.2:
            return 2
        case _ if x < -0.2:
            return 0
        case _:
            raise ValueError(f"Cant find label for {x}")

In [63]:
df["sentiment"] = (
    df["vader_comp"].apply(lambda x: get_label_value(x))
)

In [64]:
df['sentiment'].value_counts()

sentiment
2    21945
1    11893
0     7284
Name: count, dtype: int64

In [65]:
MAX_WORDS = 5000

In [66]:
tokenizer = Tokenizer(num_words=MAX_WORDS, lower=True, split=" ")
tokenizer.fit_on_texts(df["sentance"])

In [67]:
x = tokenizer.texts_to_sequences(df["sentance"])

In [68]:
y = df["sentiment"].to_numpy()
y

array([2, 2, 2, ..., 2, 0, 2])

In [69]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [70]:
num_classes = max(y) + 1

In [71]:
X_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
X_test = tokenizer.sequences_to_matrix(x_test, mode="binary")

Y_train = tf.keras.utils.to_categorical(y_train, 3)
Y_test = tf.keras.utils.to_categorical(y_test, 3)

In [72]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

X_train shape: (32897, 5000)
X_test shape: (8225, 5000)
Y_train shape: (32897, 3)
Y_test shape: (8225, 3)


In [74]:
import pickle
import json

with open("data/X_train.pickle", "wb") as X_train_file, open(
    "data/X_test.pickle", "wb"
) as X_test_file, open("data/Y_train.pickle", "wb") as Y_train_file, open(
    "data/Y_test.pickle", "wb"
) as Y_test_file, open(
    "data/vocabulary.json", "wt"
) as vocabulary_file:
    pickle.dump(X_train, X_train_file)
    pickle.dump(X_test, X_test_file)
    pickle.dump(Y_train, Y_train_file)
    pickle.dump(Y_test, Y_test_file)
    json.dump(tokenizer.word_index, vocabulary_file)